In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
import datetime
from plotly.subplots import make_subplots

In [3]:
df = pd.read_csv('..\\data\\NYPD_Complaint_Data_Historic.csv',
                 low_memory= False,
                #  nrows= 10000
                 )

### [Dataset](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i) Description


| Column Name | Description |
| ----------- | ----------- |
| CMPLNT_NUM | Randomly generated persistent ID for each complaint | 
| ADDR_PCT_CD | The precinct in which the incident occurred BORO The name of the borough in which the incident occurred |
| CMPLNT_FR_DT | Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists) |
| CMPLNT_FR_TM | Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists) |
| CMPLNT_TO_DT | Ending date of occurrence for the reported event, if exact time of occurrence is unknown |
| CMPLNT_TO_TM | Ending time of occurrence for the reported event, if exact time of occurrence is unknown |
| CRM_ATPT_CPTD_CD | Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely |
| HADEVELOPT | Name of NYCHA housing development of occurrence, if applicable |
| HOUSING_PSA | Development Level Code |
| JURISDICTION_CODE | Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc. |
| JURIS_DESC |  Description of the jurisdiction code |
| KY_CD | Three digit offense classification code |
| LAW_CAT_CD | Level of offense: felony, misdemeanor, violation |
| LOC_OF_OCCUR_DESC | Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of |
| OFNS_DESC | Description of offense corresponding with key code |
| PARKS_NM | Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included) |
| PATROL_BORO | The name of the patrol borough in which the incident occurred |
| PD_CD | Three digit internal classification code (more granular than Key Code) |
| PD_DESC | Description of internal classification corresponding with PD code (more granular than Offense Description) |
| PREM_TYP_DESC |  Specific description of premises; grocery store, residence, street, etc. |
| RPT_DT | Date event was reported to police  |
| STATION_NAME | Transit station name |
| SUSP_AGE_GROUP | Suspect’s Age Group |
| SUSP_RACE | Suspect’s Race Description |
| SUSP_SEX | Suspect’s Sex Description |
| TRANSIT_DISTRICT | Transit district in which the offense occurred. |
| VIC_AGE_GROUP | Victim’s Age Group |
| VIC_RACE | Victim’s Race Description |
| VIC_SEX | Victim’s Sex Description (D=Business/Organization, E=PSNY/People of the State of New York, F=Female, M=Male) |
| X_COORD_CD | X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |
| Y_COORD_CD | Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104) |
| Latitude | Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |
| Longitude | Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326) |

### Analysis

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

#### changing column data types

In [ ]:
# for col in df.columns:
#     print(f'\'{col}\':\'{df[col].dtype}\'', end = ', ')

# col_types = {'CMPLNT_NUM':'int64', 'CMPLNT_FR_DT': 'datetime64[ns]', 'CMPLNT_FR_TM':'datetime64[ns]', 'CMPLNT_TO_DT':'datetime64[ns]', 'CMPLNT_TO_TM':'object',
#              'ADDR_PCT_CD':'int64', 'RPT_DT':'object', 'KY_CD':'int64', 'OFNS_DESC':'object', 'PD_CD':'int64', 'PD_DESC':'object', 
#              'CRM_ATPT_CPTD_CD':'object', 'LAW_CAT_CD':'object', 'BORO_NM':'object', 'LOC_OF_OCCUR_DESC':'object', 'PREM_TYP_DESC':'object', 
#              'JURIS_DESC':'object', 'JURISDICTION_CODE':'int64', 'PARKS_NM':'int64', 'HADEVELOPT':'object', 'HOUSING_PSA':'float64', 
#              'X_COORD_CD':'float64', 'Y_COORD_CD':'float64', 'SUSP_AGE_GROUP':'object', 'SUSP_RACE':'object', 'SUSP_SEX':'object', 
#              'TRANSIT_DISTRICT':'float64', 'Latitude':'float64', 'Longitude':'float64', 'Lat_Lon':'object', 'PATROL_BORO':'object', 
#              'STATION_NAME':'float64', 'VIC_AGE_GROUP':'object', 'VIC_RACE':'object', 'VIC_SEX':'object'}

# df = df.astype(col_types)


df['CMPLNT_FR_DT'] = df['CMPLNT_FR_DT'].astype('datetime64[ns]')

#### The most and least committed crimes

In [ ]:
crime_type_count = df.groupby(['OFNS_DESC'])['OFNS_DESC'].count().sort_values(ascending= False)

# crime_type_count = {}
# for i in df.OFNS_DESC.to_list():
#     if not i in crime_type_count:
#         crime_type_count[i] = 0
#     crime_type_count[i] += 1
# crime_type_dict = dict(sorted(crime_type_dict.items(), key= lambda item: item[1], reverse= True))

crime_type_count

#### Tell me the most and least crime neighborhoods

In [ ]:
# OPTION 1: use borough
df.groupby(['BORO_NM'])['BORO_NM'].count().sort_values(ascending= False)

# OPTION 2: possibly use an external dataset of all precinct neighborhood locations, map to the complaint data


#### Tell me the stations with the most crimes

#### Most and Least commited crimes by season

In [ ]:
df_ = df[df['CMPLNT_FR_DT'] >= '01/01/2020']
df_ = df_[['CMPLNT_FR_DT', 'OFNS_DESC']].sort_values(by = 'CMPLNT_FR_DT')

In [ ]:
# sns.lineplot(df_, x= )

#### Treemap

In [ ]:
# Credit to FEDI BEN MESSAOUD: https://www.kaggle.com/code/fedi1996/boston-crime-analysis-with-plotly
def treemap(categories,title,path,values):
    fig = px.treemap(categories, path=path, values=values, height=1000,
                 title=title, color_discrete_sequence = px.colors.sequential.RdBu)
    fig.data[0].textinfo = 'label+text+value'
    fig.show()

crime_count = df['OFNS_DESC'].value_counts()
vals = crime_count.values

categories = pd.DataFrame(data= crime_count.index, columns=["OFNS_DESC"])
categories['values'] = vals

treemap(categories,'Crimes in New York',['OFNS_DESC'],categories['values'])